In [1]:
import sys
sys.path.append("../../")
import os
import numpy as np
import pandas as pd
import datetime
import math
import zipfile
import re
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from deeprec_utils import download_deeprec_resources 
from newsrec_utils import prepare_hparams
from mind_iterator import MINDIterator
from newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

c:\users\v-yianwang\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\v-yianwang\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\v-yianwang\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\v-yianwang\anaconda3\envs\py36\lib\site-packages\ten

ModuleNotFoundError: No module named 'reco_utils'

In [2]:
epochs = 5
seed = 40
batch_size = 32

# Options: demo, small, large
MIND_type = 'small'

In [10]:
data_path = r'MIND/small'


train_news_file = os.path.join(data_path, 'MINDsmall_train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'MINDsmall_train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'MINDsmall_dev', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'MINDsmall_dev', r'behaviors.tsv')


In [11]:
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

"""mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)"""

"mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)\n\nif not os.path.exists(train_news_file):\n    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)\n    \nif not os.path.exists(valid_news_file):\n    download_deeprec_resources(mind_url,                                os.path.join(data_path, 'valid'), mind_dev_dataset)\nif not os.path.exists(yaml_file):\n    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/',                                os.path.join(data_path, 'utils'), mind_utils)"

### Data preprocessing

In [6]:
His = 10 # Maximum number of news


In [7]:
valid_news_file

'\\MIND\\small\\MINDsmall_dev\\news.tsv'

In [8]:
def read_behaviors(file):
    new_col = ['impr_id', 'uid', 'time', 'history', 'impressions']
    behaviors = pd.read_csv(file, names = new_col, sep='\t', header=None)
    return behaviors
def read_news(file):
    new_col = ['nid', 'category', 'subcategory', 'title', 'abs', 'url', 'title_entities', 'abs_entities']
    news = pd.read_csv(valid_news_file, names = new_col, sep='\t', header=None)
    return news

In [12]:
valid_behaviors = read_behaviors(valid_behaviors_file)
train_behaviors = read_behaviors(train_behaviors_file)

valid_news = read_news(valid_news_file)
train_news = read_news(train_news_file)

In [28]:
u2index = pd.read_pickle(userDict_file)

In [193]:
valid_news.loc[2389]

nid                                                          N49553
category                                                  lifestyle
subcategory                         lifestylefamilyandrelationships
title                      The 50 Most Common Last Names in America
abs                                               What's in a name?
url                   https://assets.msn.com/labs/mind/AAGO6bg.html
title_entities    [{"Label": "Surname", "Type": "C", "WikidataId...
abs_entities                                                     []
Name: 2389, dtype: object

In [238]:
def get_entities(data):
    if data == '[]':
        return []
    str2list = re.findall("\{.*?\}", data)
    num = len(str2list)
    list2dict = []
    for i in range(num):
        list2dict.append(eval(str2list[i]))
    return list2dict

def aggregate(title, abstract):
    flag = []
    for i in range(len(title)):
        flag.append(title[i]['WikidataId'])
    for i in range(len(abstract)):
        if abstract[i]['WikidataId'] in flag:
            continue
        else:
            title.append(abstract[i])
    return title

def news2entities(df):
    all_entities = []
    for i in range(len(df)):
        #print(i)
        title = get_entities(df['title_entities'][i])
        abstract = get_entities(df['abs_entities'][i])
        
        all_entities.append(aggregate(title, abstract))
        return all_entities

In [239]:
new_column = news2entities(valid_news)

In [244]:
valid_news['all_entities'] = all_entities

### remove unfound history

In [74]:
for i in range(38, len(valid_behaviors)):
    if type(valid_behaviors['history'][i]) == str:
        if 'N18259' in valid_behaviors['history'][i]:
            valid_behaviors['history'][i] = valid_behaviors['history'][i].replace('N18259', '')
    else: 
        continue

c:\users\v-yianwang\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [78]:
valid_behaviors.to_csv('test.tsv', sep='\t', header=None, index=None)

In [79]:
from tensorflow import keras

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers


In [14]:
model = keras.Sequential()
model.add(layers.TimeDistributed(layers.Dense(400, input_shape=(30, )), input_shape=(None, 50, 30)))

In [15]:
model.output_shape

(None, None, 50, 400)

In [ ]:
Dense(32, input_shape=(16,))